# Covid-vaccinationsprogrammet
### Välkommen - följ instruktionerna nedan

1. I menyn "Run", välj "Run all cells" (längst ner i menyn).
2. Scrolla längst ner i det här dokumentet till rubriken "Använd programmet här"

<img src="https://res.cloudinary.com/practicaldev/image/fetch/s--4nVcu5jx--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_66%2Cw_880/https://cdn.dribbble.com/users/722835/screenshots/4082720/bot_icon.gif" style="max-width:500px;" align="left">

In [1]:
from pprint import pprint
import os
import re
import time
from time import sleep

from pynput.mouse import Button, Controller as MouseCtrlr
from pynput.keyboard import Key, Controller as KeyCtrlr
import pyautogui as pag
import keyboard as keyboard


from matplotlib import pyplot as plt
from PIL import Image
import cv2 as cv
import numpy as np

In [2]:
# import pytesseract as tess
# tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [3]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
from termcolor import colored, cprint

# Helper functions

In [4]:
def imshow(img, cmap=None, size=10):
    if type(img) == str:
        img = cv.imread(img)
    elif(not type(img) == np.ndarray):
        return
    
    fig = plt.figure(figsize=(size,size))
    ax = plt.Axes(fig, [0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(img, cmap)

In [5]:
def get_inverted(image):
    return cv.bitwise_not(image)

# get grayscale image
def get_grayscale(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

#thresholding
def thresholding(image, threshold=180):
    return cv.threshold(image, threshold, 255, cv.THRESH_BINARY)[1]
    #return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    
def thresholdingAdaptive(image):
    return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)

In [6]:
kb = KeyCtrlr()

In [7]:
class Pos:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __bool__(self):
        return True

In [8]:
def message(str):
    return pag.confirm(str)

In [9]:
globalWaitTime = 1
globalLimitTime = 10
globalWaitBeforeStart = 0.2

def getWaitAndLimit(beforeStart = -1, limit = -1, wait = -1):
    if wait == -1:
        wait = globalWaitTime
    if limit == -1:
        limit = globalLimitTime
    if beforeStart == -1:
        beforeStart = globalWaitBeforeStart
        
    return (beforeStart, limit, wait)

In [10]:
def waitForElement(path, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
        
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
            return Pos(x, y)
        except Exception as e:
            pass
            #print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element " + path)
    return False
        
#waitForElement("SokValjPatient")

In [11]:
def waitForElementDisappear(path, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
        except Exception as e:
            if (str(e) == "cannot unpack non-iterable NoneType object"):
                return True
            else:
                print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element disappear " + path)
    return False


In [12]:
def waitForOneOf(pathList, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        for path in pathList:
            pos = waitForElement(path, beforeStart = 0, limit = 0.01, waitTime = 0.01, conf = conf, warn = False)
            timePassed += 0.01
            if pos:
                return pos
            
        sleep(waitTime)
        timePassed += waitTime
        
    if warn:
        message("Timed out waiting for element list " + str(pathList))
        
    return False
    
#waitForOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"], limit=2)

In [13]:
def waitAndClickOneOf(pathList, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button = "left"):
    pos = waitForOneOf(pathList, beforeStart = beforeStart, limit = limit, waitTime = waitTime, conf = conf, warn = warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
            
        return pos
    
    return False


#waitAndClickOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"])

In [14]:
def waitAndClickElement(path, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button="left"):
    pos = waitForElement(path, beforeStart, limit, waitTime, conf, warn=warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        #print(f'Moving to {pos.x}, {pos.y}')
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
        elif button == "double":
            pag.doubleClick(pos.x, pos.y)
        return pos
    else:
        return False

#waitAndClickElement("SokValjPatient")

In [15]:
def waitAndClickXY(x, y, beforeStart = -1, waitTime = -1, button="left"):
    pos = Pos(x, y)
    return waitAndClick([pos], beforeStart = beforeStart, waitTime = waitTime, button=button)

def waitAndClick(positions, beforeStart = -1, waitTime = -1, button="left"):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    try:
        if type(positions) != list and type(positions) != tuple:
            positions = [positions]

        #print(f"Sleeping {beforeStart} before starting")
        sleep(beforeStart)
        for i, pos in enumerate(positions):
            if type(pos) == list or type(pos) == tuple:
                posObj = Pos(pos[0], pos[1])
            else:
                posObj = pos
            
            #print(text)
            if button == "left":
                pag.leftClick(posObj.x, posObj.y)
            elif button == "right":
                pag.rightClick(posObj.x, posObj.y)
            if i != len(positions) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndClick: " + str(e))
        return False
    

In [16]:
def typeSlowly(text, keyInterval):
    for ch in text:
        kb.type(ch)
        sleep(keyInterval)


In [17]:
def waitAndType(texts, beforeStart = -1, waitTime = -1, keyInterval=0):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    if keyInterval == True:
        keyInterval = 0.05
    
    try:
        if type(texts) == str:
            texts = [texts]

        sleep(beforeStart)
        for i, text in enumerate(texts):
            #print(text)
            if keyInterval <= 0:
                kb.type(text)
            else:
                typeSlowly(text, keyInterval)
                
            if i != len(texts) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndType: " + str(e))
        return False
        
        
#waitAndType(["hej", "hej2", "hej3"])
#waitAndType("hejdå")

In [18]:
def waitAndPress(key1, key2 = None, key3 = None, beforeStart = -1):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart)
    
    sleep(beforeStart)
    
    try:
        if key2 and key3:
            keyboard.press_and_release(key1+"+"+key2+"+"+key3)
        elif key2:
            keyboard.press_and_release(key1+"+"+key2)
        else:
            keyboard.press_and_release(key1)
        return True
    except Exception as e:
        pag.confirm(f"Failed wait and press {str(key1)},{str(key2)},{str(key3)}")
        return False
    

In [19]:
def AsError(text):
    return colored(text, "red")

def AsSuccess(text):
    return colored(text, "green")

In [20]:
def PrintSuccessStatus(result, title):
    outputText = title + ":"
    resultMessage = ""
    if result:
        resultMessage = AsSuccess("Lyckades!")
    else:
        resultMessage = AsError("MISSLYCKADES!")
    print(outputText, resultMessage)
    return result

# Complex function

In [34]:
class ImgsEnum:
    def __init__(self, imagesPath):
        self.ArbetsflodeLabel = os.path.join(imagesPath, "Vaccine", "ArbetsflodeLabel.png")
        self.StangButtonText = os.path.join(imagesPath, "Vaccine", "StangButtonText.png")
        self.NyDiagnosTab = os.path.join(imagesPath, "Vaccine", "NyDiagnosTab.png")
        self.KshDropdownChoice = os.path.join(imagesPath, "Vaccine", "KshDropdownChoice.png")
        self.KshDropdownChoice2 = os.path.join(imagesPath, "Vaccine", "KshDropdownChoice2.png")
        self.Icd10DropdownChoice = os.path.join(imagesPath, "Vaccine", "icd10DropdownChoice.png")
        self.IcdSearchInputField = os.path.join(imagesPath, "Vaccine", "icdSearchInputField.png")
        self.NyVaccinationSearchField = os.path.join(imagesPath, "Vaccine", "NyVaccinationSearchField.png")
        self.NyAtgardTab = os.path.join(imagesPath, "Vaccine", "NyAtgardTab.png")
        self.Covid19AtgardChoice = os.path.join(imagesPath, "Vaccine", "covid19AtgardChoice.png")
        self.VaccinationCovid19Choice = os.path.join(imagesPath, "Vaccine", "VaccinationCovid19Choice.png")
        self.AtgardRelateradCovid19 = os.path.join(imagesPath, "Vaccine", "AtgardRelateradCovid19.png")
        self.CitrixIcon = os.path.join(imagesPath, "Vaccine", "CitrixIcon.png")
        self.CitrixIcon2 = os.path.join(imagesPath, "Vaccine", "CitrixIcon2.png")
        self.LotNrLabel = os.path.join(imagesPath, "Vaccine", "LotNrLabel.png")
        self.LotNrLabel2 = os.path.join(imagesPath, "Vaccine", "LotNrLabel2.png")
        self.LotNrLabel3 = os.path.join(imagesPath, "Vaccine", "LotNrLabel3.png")
        
        self.DosNummerInputField = os.path.join(imagesPath, "Vaccine", "DosNummerInputField.png")
        self.KontraindikationNejCheckbox = os.path.join(imagesPath, "Vaccine", "KontraindikationNejCheckbox.png")
        self.NyVardkontaktTitle = os.path.join(imagesPath, "Vaccine", "NyVardkontaktTitle.png")
        self.Covid19VaccinationPatientCheckbox = os.path.join(imagesPath, "Vaccine", "Covid19VaccinationPatientCheckbox.png")
        self.VaccinationsmottagningCheckbox = os.path.join(imagesPath, "Vaccine", "VaccinationsmottagningCheckbox.png")
        self.NyDiagnosTitle = os.path.join(imagesPath, "Vaccine", "NyDiagnosTitle.png")
        self.NyAtgardTitle = os.path.join(imagesPath, "Vaccine", "NyAtgardTitle.png")
        self.NyVaccinationTab = os.path.join(imagesPath, "Vaccine", "NyVaccinationTab.png")
        self.SigneraDisabeledMenu = os.path.join(imagesPath, "Vaccine", "SigneraDisabeledMenu.png")
        self.SokTabIcdSearch = os.path.join(imagesPath, "Vaccine", "SokTabIcdSearch.png")
        
        self.VaccinationerAtgardChoice = os.path.join(imagesPath, "Vaccine", "VaccinationerAtgardChoice.png")
        self.InfluensasAtgardChoice = os.path.join(imagesPath, "Vaccine", "InfluensasAtgardChoice.png")
        self.InfluensaPneumokockerAtgardChoice = os.path.join(imagesPath, "Vaccine", "InfluensaPneumokockerAtgardChoice.png")
        self.NyVaccinationButton = os.path.join(imagesPath, "Vaccine", "NyVaccinationButton.png")
        self.LokalisationLabel =  os.path.join(imagesPath, "Vaccine", "LokalisationLabel.png")
        self.LokalisationLabel2 = os.path.join(imagesPath, "Vaccine", "LokalisationLabel2.png")
        self.VaccinationLeftMenuItem = os.path.join(imagesPath, "Vaccine", "VaccinationLeftMenuItem.png")
        self.NyVaccinationTitle = os.path.join(imagesPath, "Vaccine", "NyVaccinationTitle.png")
        

        
        #self. = os.path.join(imagesPath, "Vaccine", ".png")

    
IMG = ImgsEnum(os.path.join("..", "Images"))

IMG.ArbetsflodeLabel


'..\\Images\\Vaccine\\ArbetsflodeLabel.png'

In [22]:
def SignOrSave():
    foundIt = waitForElement(IMG.SigneraDisabeledMenu, limit=0.2, warn=False)
    if not foundIt:
        waitAndPress("alt", "g")
    else:
        waitAndPress("alt", "s")


In [23]:
def OpenPatient():
    pos = waitAndClickElement(IMG.ArbetsflodeLabel, relY=18, conf=0.8)
    if pos:
        waitAndType("covid")
        waitAndPress("enter")
        waitAndPress("enter")
        
        timer = 0
        
        readyPos = False
        while timer < 15 and not readyPos:
            readyPos = waitForElement(IMG.NyVaccinationSearchField, limit=2, warn=False)
            if not readyPos:
                closePos = waitAndClickElement(IMG.StangButtonText, limit=0.2, warn=False)
                if closePos:
                    waitAndClickElement(IMG.StangButtonText, beforeStart=0.5, limit=0.2, warn=False)
    
    return pos

In [24]:
def RegisterVisit(tasks):
    waitAndPress("ctrl", "e")
    pos = waitForElement(IMG.NyVardkontaktTitle)
    if pos:
        sleep(0.3)
        waitAndType("m")
    
    if "covid" in tasks:
        if pos:
            pos = waitAndClickElement(IMG.Covid19VaccinationPatientCheckbox, relX=-81)
    if pos:
        pos = waitAndClickElement(IMG.VaccinationsmottagningCheckbox, relX=-71)
        
    if pos:
        SignOrSave()
    if pos:
        pos = waitForElementDisappear(IMG.NyVardkontaktTitle)
        
    return pos


In [25]:
def SelectAndClearDiagnosisSearchField():
    pos = waitAndClickElement(IMG.SokTabIcdSearch, relY=30)
    
    if pos:
        waitAndPress("ctrl", "A")
        waitAndPress("delete")
        
    return pos



In [26]:
def SetDiagnosis(tasks):
    pos = waitAndClickElement(IMG.NyDiagnosTab)
    if pos:
        kshPos = waitAndClickOneOf([IMG.KshDropdownChoice, IMG.KshDropdownChoice2], conf=0.75, limit=0.5, warn=False)
        if kshPos:
            waitAndType("icd")
            waitAndPress("enter")
    
    if pos and "covid" in tasks:
        pos = SelectAndClearDiagnosisSearchField()
        if pos:
            waitAndType("U119")
            waitAndPress("enter")
            waitAndPress("enter")
            
    if pos and "influensa" in tasks:
        pos = SelectAndClearDiagnosisSearchField()
        if pos:
            waitAndType("Z251")
            waitAndPress("enter")
            waitAndPress("enter")
    
    if pos:
        SignOrSave()
    if pos:
        pos = waitForElementDisappear(IMG.NyDiagnosTitle)
    return pos


In [27]:
def SetActionCode(tasks):
    pos = waitAndClickElement(IMG.NyAtgardTab)
    
    if "covid" in tasks:
        if pos:
            pos = waitAndClickElement(IMG.Covid19AtgardChoice, relX=-31)
            if "influensa" in tasks:
                pag.scroll(-500)
                sleep(0.2)
        if pos:
            pos = waitAndClickElement(IMG.VaccinationCovid19Choice, button="double")
        if pos:
            pos = waitAndClickElement(IMG.AtgardRelateradCovid19, button="double")
            
    if "influensa" in tasks:
        pos = waitAndClickElement(IMG.VaccinationerAtgardChoice, relX=-41)
        pag.scroll(-1000)
        sleep(0.2)
        if pos and "pneumokocker" in tasks:
            pos = waitAndClickElement(IMG.InfluensaPneumokockerAtgardChoice, button="double")
        elif pos:
            pos = waitAndClickElement(IMG.InfluensasAtgardChoice, button="double")
        
    if pos:
        SignOrSave()
        
    if pos:
        pos = waitForElementDisappear(IMG.NyAtgardTitle)
        
    return pos


In [37]:
def SelectInjectionSite(site = "vä ö"):
    pos = waitAndClickOneOf([IMG.LokalisationLabel, IMG.LokalisationLabel2], limit=2, relY=17, conf=0.8)
    if pos:
        waitAndType(site)
        waitAndPress("enter")
        
    return pos
    
pos = PrintSuccessStatus(waitAndClickOneOf([IMG.CitrixIcon, IMG.CitrixIcon2]), " - Öppnar PMO")
if pos:
    print(SelectInjectionSite())

 - Öppnar PMO: Lyckades!
(299, 425)


In [29]:
def RegisterEnskildVaccination(vaccineName, doseNo, batchNo, site):
    
    waitAndType(vaccineName, keyInterval=True)
    sleep(0.2)
    waitAndPress("enter")
    
    pos = SelectInjectionSite(site)
    if pos:
        pos = waitAndClickOneOf([IMG.LotNrLabel,IMG.LotNrLabel2, IMG.LotNrLabel3], relY=18)
    if pos:
        waitAndPress("ctrl", "a")
        waitAndPress("delete")
        waitAndType(batchNo)
    if pos:
        pos = waitAndClickElement(IMG.DosNummerInputField)
    if pos:
        waitAndType(doseNo)
    if pos:
        pos = waitAndClickElement(IMG.KontraindikationNejCheckbox)
    
    return pos



In [30]:
def CreateNewVaccination():
    pos = waitAndClickElement(IMG.NyVaccinationButton, conf=0.98)
    return pos


In [42]:
def RegistreraVaccinationer(vaccinationDataList):
    first = True
    pos = True
    
    for vaccinationData in vaccinationDataList:
        if pos:
            if first:
                pos = waitAndClickElement(IMG.NyVaccinationTab)
                first = False
            else:
                pos = CreateNewVaccination()
            
            if pos:
                pos = waitForElement(IMG.NyVaccinationSearchField)

            pos = RegisterEnskildVaccination(vaccinationData["Name"], vaccinationData["DoseNo"], vaccinationData["BatchNo"], vaccinationData["site"])
            
            if pos and len(vaccinationDataList) > 1:
                waitAndPress("alt", "s")
                pos = waitForElementDisappear(IMG.NyVaccinationTitle)
            
    if pos and len(vaccinationDataList) > 1:
        waitAndClickElement(IMG.VaccinationLeftMenuItem, button="double")

# pos = PrintSuccessStatus(waitAndClickOneOf([IMG.CitrixIcon, IMG.CitrixIcon2]), " - Öppnar PMO")
# if pos:
#     RegistreraVaccinationer([
#         {"tag": "covid", "Name": "Covid-19 vaccine Moder", "DoseNo": "2", "BatchNo": "ZXY789", "site": "hö ö"}
#     ])

 - Öppnar PMO: Lyckades!


In [43]:
def RunFullVaccinationSequence(vaccinationDataList):
    
    print("Registrerar med...");
    
    tasks = []
    
    for data in vaccinationDataList:
        tasks.append(data["tag"])
        print(f"""Vaccin: {data["Name"]}
Batch: {data["BatchNo"]}
Dos: {data["DoseNo"]}

""")
        
    if data["batchNo"] == "":
        print(AsError("FEL - Du måste ange ett batchnummer"))
        return
    
    finishMessage = ""
    
    finishMessage = "Misslyckades med att öppna PMO. Är det startat?"
    
    pos = PrintSuccessStatus(waitAndClickOneOf([IMG.CitrixIcon, IMG.CitrixIcon2]), " - Öppnar PMO")
    if pos:
        pos = PrintSuccessStatus(OpenPatient(), " - Öppnar patienten")
        finishMessage = "Du måste dubbelklicka på rätt patient i tidboken innan du kör programmet."
    if pos:
        pos = PrintSuccessStatus(RegisterVisit(tasks), " - Besökregistrering")
        finishMessage = "Du måste själv komplettera med det som inte blev gjort. Du kan sedan köra programmet på nästa patient."
    if pos:
        pos = PrintSuccessStatus(SetDiagnosis(tasks), " - Sätter diagnos")
    if pos:
        pos = PrintSuccessStatus(SetActionCode(tasks), " - Sätter KVÅ")
    if pos:
        pos = PrintSuccessStatus(RegistreraVaccinationer(vaccinationDataList), " - Registrerar vaccin")
        
        
    if pos:
        print(AsSuccess("\nAVSLUTAR"))
        finishMessage = "Redo för en ny omgång."
    else:
        print(AsError("\nAVSLUTAR"))
        
    print("\n - " + finishMessage)


# User interface

In [48]:
vaccineDropdownCovid =  widgets.Dropdown(
                        options=['AstraZeneca', 'Moderna', 'Janssen', "Pfizer (Comirnaty)"],
                        value='AstraZeneca',
                        description='Välj vaccin:')

vaccineDictCovid = {
    'AstraZeneca': "covid-19 vaccine ast",
    'Moderna': "covid-19 vaccine mod",
    'Janssen': "covid-19 vaccine jan",
    "Pfizer (Comirnaty)": "comirnaty"
}


vaccineDropdownInfluensa =  widgets.Dropdown(
                        options=['Influvac Tetra', 'Vaxigrip Tetra'],
                        value='Influvac Tetra',
                        description='Välj vaccin:')

vaccineDictInfluensa = {
    'Influvac Tetra': "Influvac T",
    'Vaxigrip Tetra': "Vaxigrip T",
}


doseDropdownCovid = widgets.Dropdown(
               options=['Dos 1', 'Dos 2', "Dos 3"],
               value='Dos 1',
               description='Välj dos:')

doseDict = {
    "Dos 1": "1",
    "Dos 2": "2",
    "Dos 3": "3"
}

batchTextFieldCovid = widgets.Text(
                      value='',
                      description='Skriv batchnummer', )

batchTextFieldInfluensa = widgets.Text(
                      value='',
                      description='Skriv batchnummer', )

batchTextFieldPneumo = widgets.Text(
                      value='',
                      description='Skriv batchnummer', )

startButtonCovid = widgets.Button(description='Starta Covid-19', 
                             style=widgets.ButtonStyle(button_color="rgb(40,80,200)"))
startButtonInfluensa = widgets.Button(description='Starta Influensa', 
                             style=widgets.ButtonStyle(button_color="rgb(40,80,200)"))
startButtonCovidInflu = widgets.Button(description='Starta Covid+Influensa', 
                             style=widgets.ButtonStyle(button_color="rgb(40,80,200)"))
startButtonInfluensaPneumo = widgets.Button(description='Starta Influensa+Pneumokock', 
                             style=widgets.ButtonStyle(button_color="rgb(40,80,200)"))

outputArea =  widgets.Output()


covidLabel = widgets.HTML(
    value="<h2>Covid-19</h2>",
    placeholder='Covid-19',
)

influensaLabel = widgets.HTML(
    value="<h2>Influensa</h2>",
    placeholder='Influensa',
)

pneumoLabel = widgets.HTML(
    value="<h2>Pneumokocker</h2>",
    placeholder='Influensa',
)

runLabel = widgets.HTML(
    value="<h2>Kör programmet</h2>",
    placeholder='Kör programmet',
)

covidColumn = widgets.VBox([covidLabel,
                            vaccineDropdownCovid, 
                            doseDropdownCovid, 
                            batchTextFieldCovid, ])

pneumoColumn = widgets.VBox([pneumoLabel,
                             batchTextFieldPneumo])

influensaColumn = widgets.VBox([influensaLabel,
                                vaccineDropdownInfluensa,
                                batchTextFieldInfluensa,
                                pneumoColumn])

inputArea = widgets.HBox(  [covidColumn,
                            influensaColumn])

runArea = widgets.VBox(    [runLabel,
                            startButtonCovid,
                            startButtonInfluensa,
                            startButtonCovidInflu,
                            startButtonInfluensaPneumo])

outputColumn = widgets.VBox( [outputArea])

runAndOutputArea = widgets.HBox ([runArea, 
                                  outputColumn])


Interface = widgets.VBox(  [inputArea,
                            runAndOutputArea])

def onCovid(_):
    vaccinationDataList = [{
        "tag": "covid",
        "Name": ,
        "DoseNo": ,
        "BatchNo": ,
    }]
    
    run(vaccinationDataList)
        
def onInfluensa(_):
    vaccinationDataList = [{
        "tag": ,
        "Name": ,
        "DoseNo": ,
        "BatchNo": ,
    }]
    
    run(vaccinationDataList)
        
def onCovidInflu(_):
    vaccinationDataList = [{
        "tag": ,
        "Name": ,
        "DoseNo": ,
        "BatchNo": ,
    }]
    
    run(vaccinationDataList)
        
def onInfluensaPneumo(_):
    vaccinationDataList = [{
        "tag": ,
        "Name": ,
        "DoseNo": ,
        "BatchNo": ,
    }]
    
    run(vaccinationDataList)

def run(vaccinationDataList):
    # "linking function with output"
    with outputArea:
        # what happens when we press the button
        clear_output()
        RunFullVaccinationSequence(vaccinationDataList)
        
# linking button and function together using a button's method
startButtonCovid.on_click(onCovid)
startButtonInfluensa.on_click(onInfluensa)
startButtonCovidInflu.on_click(onCovidInflu)
startButtonInfluensaPneumo.on_click(onInfluensaPneumo)


<br/>
<br/>
<img src="https://media.istockphoto.com/vectors/cute-cartoon-car-set-with-dash-line-and-horizontal-road-baby-kids-vector-id1141945630" style="max-width:500px;display:block;" align="left">

# Använd programmet här!
1. Välj vaccin, dos och skriv in batch nummer för de vaccin som är relevanta.
2. Dubbelklicka på rätt patient i tidboken i PMO, men gå inte in i journalen.
3. Om du inte själv är sjuksköterska eller läkare, välj att du arbetar rollen som sjuksköterska
4. Klicka på Starta för de vaccin du ska ge.
5. När du är klar med en patient kan du börja om från nr 2 ovan med nästa patient.

**Notera:** Om fler än ett vaccin används på samma gång, sparas de. Du måste du själv kontrollera vaccinen i vaccinationslistan och signera dem (med alt+g) om inget ska ändras.

In [49]:
Interface

<img src="https://cdn.pixabay.com/photo/2014/10/19/04/52/light-494011__340.png" style="max-width:500px;display:block;" align="left">
